In [1]:
import os,sys,string, time
import ROOT
from math import *
from ROOT import gPad, TTree, TObject, TFile, gDirectory, TH1D, TH2D, TH3D, TCanvas, gROOT, TGaxis, gStyle, TColor, TLegend, THStack, TChain, TLatex, TText, TCollection, kRed, kBlue
from array import array
import matplotlib.pyplot as plt
import numpy as np
import uproot
import pandas as pd
from root_pandas import read_root
from platform import python_version
import scipy
from scipy import stats
import uproot3
import math
from matplotlib.patches import Rectangle
import pickle
import functools
from importlib import reload

import Utilities.Plotter as PT
import Utilities.Constants as Constants
import Utilities.Variables_list as Variables
import Utilities.Functions as Functions
from Utilities.ipython_exit import exit

print ('Success')


Welcome to JupyROOT 6.24/06
Success


# Setting which samples to pickle

In [2]:
Params = {"Run":"run3", #The run number, so far either "run1" or "run3"
          "Load_single_file":False, #This will override everything else, put the desired file in the "single_file" line
          "single_file":'beamgood',
          "Load_lepton_signal":False, #Just e+e- samples
          "Load_standard_bkgs":False, #Backgrounds
          "Load_DetVars":False,
          "Only_keep_common_DetVar_evs":True, #For overlay only atm
          "Load_Signal_DetVars":False,
          "Load_data":False,
          "FLATTEN":True, #Have one row per reconstructed object in the analysis dataframe
          "only_presel":False,#Create small files containing only variables necessary for pre-selection, for making pre-selection plots
          "Load_truth_vars":False, #Load the "mc_" type variables for signal
          "Load_pi0_signal":False,
          "Load_pi0_signal_DetVars":True,
          "Load_sys_vars":False} #Load in the signal samples where the HNL decays to pi0


In [3]:
Params, samples = Functions.create_sample_list(Params)

sample_loc = {"overlay":f'../NuMI_MC/SLIMMED_neutrinoselection_filt_'+Params["Run"]+'_overlay.root',
              "dirtoverlay":f'../NuMI_MC/neutrinoselection_filt_'+Params["Run"]+'_dirt_overlay.root',
              "beamoff":f'../NuMI_data/neutrinoselection_filt_'+Params["Run"]+'_beamoff.root',
              "signal":f'../NuMI_signal/KDAR_dump/sfnues/sfnues_KDAR_dump_',
              "pi0_signal":f'../NuMI_signal/KDAR_dump/sfnues/pi0/sfnues_KDAR_dump_',
              "beamgood":f'../NuMI_data/neutrinoselection_filt_'+Params["Run"]+'_beamon_beamgood.root'}

root_dir = 'nuselection'
main_tree = 'NeutrinoSelectionFilter'
loc_pkls = "pkl_files/"+Params["Run"]+"/current_files/"+Params["variables_string"]+"/"


Loading these run3 samples: 
['150_WireModX', '150_WireModYZ', '150_WireModThetaXZ', '150_WireModThetaYZ', '150_WireModdEdX', '150_LYDown', '150_LYRayleigh', '150_LYAttenuation', '150_SCE', '150_Recomb2', '150_CV', '180_WireModX', '180_WireModYZ', '180_WireModThetaXZ', '180_WireModThetaYZ', '180_WireModdEdX', '180_LYDown', '180_LYRayleigh', '180_LYAttenuation', '180_SCE', '180_Recomb2', '180_CV', '200_WireModX', '200_WireModYZ', '200_WireModThetaXZ', '200_WireModThetaYZ', '200_WireModdEdX', '200_LYDown', '200_LYRayleigh', '200_LYAttenuation', '200_SCE', '200_Recomb2', '200_CV', '220_WireModX', '220_WireModYZ', '220_WireModThetaXZ', '220_WireModThetaYZ', '220_WireModdEdX', '220_LYDown', '220_LYRayleigh', '220_LYAttenuation', '220_SCE', '220_Recomb2', '220_CV', '240_WireModX', '240_WireModYZ', '240_WireModThetaXZ', '240_WireModThetaYZ', '240_WireModdEdX', '240_LYDown', '240_LYRayleigh', '240_LYAttenuation', '240_SCE', '240_Recomb2', '240_CV', '245_WireModX', '245_WireModYZ', '245_WireMod

In [4]:
print(Params["variables_MC"])

['run', 'sub', 'evt', 'nslice', 'n_pfps', 'n_tracks', 'n_showers', 'swtrig_pre', 'swtrig_post', 'trk_sce_start_x_v', 'trk_sce_start_y_v', 'trk_sce_start_z_v', 'trk_sce_end_x_v', 'trk_sce_end_y_v', 'trk_sce_end_z_v', 'shr_theta_v', 'shr_phi_v', 'shr_px_v', 'shr_py_v', 'shr_pz_v', 'shrclusdir0', 'shrclusdir1', 'shrclusdir2', 'shr_energy_tot', 'trk_theta_v', 'trk_phi_v', 'trk_dir_x_v', 'trk_dir_y_v', 'trk_dir_z_v', 'trk_energy', 'trk_energy_hits_tot', 'trk_energy_tot', 'trk_score_v', 'trk_calo_energy_u_v', 'trk_end_x_v', 'trk_chipr_best', 'pfnplanehits_U', 'pfnplanehits_V', 'pfnplanehits_Y', 'NeutrinoEnergy2', 'SliceCaloEnergy2', 'nu_flashmatch_score', 'contained_sps_ratio', 'flash_time', 'contained_fraction', 'trk_score', 'crtveto', 'weightSplineTimesTune', 'ppfx_cv', 'npi0']


In [5]:
if Params["Load_sys_vars"] == True:
    Params["variables_MC"] += Variables.sys_vars
    
print(Params["variables_MC"])

['run', 'sub', 'evt', 'nslice', 'n_pfps', 'n_tracks', 'n_showers', 'swtrig_pre', 'swtrig_post', 'trk_sce_start_x_v', 'trk_sce_start_y_v', 'trk_sce_start_z_v', 'trk_sce_end_x_v', 'trk_sce_end_y_v', 'trk_sce_end_z_v', 'shr_theta_v', 'shr_phi_v', 'shr_px_v', 'shr_py_v', 'shr_pz_v', 'shrclusdir0', 'shrclusdir1', 'shrclusdir2', 'shr_energy_tot', 'trk_theta_v', 'trk_phi_v', 'trk_dir_x_v', 'trk_dir_y_v', 'trk_dir_z_v', 'trk_energy', 'trk_energy_hits_tot', 'trk_energy_tot', 'trk_score_v', 'trk_calo_energy_u_v', 'trk_end_x_v', 'trk_chipr_best', 'pfnplanehits_U', 'pfnplanehits_V', 'pfnplanehits_Y', 'NeutrinoEnergy2', 'SliceCaloEnergy2', 'nu_flashmatch_score', 'contained_sps_ratio', 'flash_time', 'contained_fraction', 'trk_score', 'crtveto', 'weightSplineTimesTune', 'ppfx_cv', 'npi0']


In [6]:
if (Params["Load_DetVars"] == True) and (Params["Only_keep_common_DetVar_evs"] == True):
    df_rse_list = []
    print("Loading only run, subrun, event numbers" + "\n")
    for sample in samples: #Looping over all samples, should make a function for this.
        if sample in Constants.Detector_variations: #Checks if it is an overlay Detector Variation sample
            print(f"Loading overlay {sample} "+Params["Run"]+" with uproot")
            NuMI_MC_overlay = uproot3.open("../NuMI_MC/DetVars/neutrinoselection_filt_"+Params["Run"]+f"_overlay_{sample}.root")[root_dir+"/"+main_tree]
            df_overlay = NuMI_MC_overlay.pandas.df(['run','sub','evt'], flatten=Params["FLATTEN"])
            file = df_overlay
            Functions.make_unique_ev_id(file) #This creates "rse_id" branch
            new_overlay = file.copy()
            df_rse_list.append(new_overlay)

    common_evs = Functions.make_common_evs_df(df_rse_list)
    
elif (Params["Load_Signal_DetVars"] == True) and (Params["Only_keep_common_DetVar_evs"] == True): #WRONG
    common_evs_dict = {}
    print("Loading only run, subrun, event numbers" + "\n")
    if Params["Run"] == "run1": HNL_samples = [150]
    if Params["Run"] == "run3": HNL_samples = [2, 10, 20, 50, 100]
    for HNL_mass in HNL_samples: #Currently don't have 150MeV detvar samples for run3
        df_rse_list = []
        for DetVar in Constants.Detector_variations: #Looping over all samples, should make a function for this.
            #print(f"Loading overlay {sample} "+Params["Run"]+" with uproot")
            NuMI_MC_signal = uproot3.open("../NuMI_signal/KDAR_dump/sfnues/DetVars/"+f"{HNL_mass}_{DetVar}_"+Params["Run"]+".root")[root_dir+"/"+main_tree]
            df_signal = NuMI_MC_signal.pandas.df(['run','sub','evt'], flatten=Params["FLATTEN"])
            file = df_signal
            Functions.make_unique_ev_id(file) #This creates "rse_id" branch
            new_signal = file.copy()
            df_rse_list.append(new_signal)

        common_evs = Functions.make_common_evs_df(df_rse_list)
        common_evs_dict[HNL_mass] = common_evs
        
elif (Params["Load_pi0_signal_DetVars"] == True) and (Params["Only_keep_common_DetVar_evs"] == True): #WRONG
    common_evs_dict = {}
    print("Loading only run, subrun, event numbers" + "\n")
    if Params["Run"] == "run1": HNL_samples = []
    if Params["Run"] == "run3": HNL_samples = [150,180,200,220,240,245]
    for HNL_mass in HNL_samples: #Currently don't have 150MeV detvar samples for run3
        print(HNL_mass)
        df_rse_list = []
        for DetVar in Constants.Detector_variations: #Looping over all samples, should make a function for this.
            #print(f"Loading overlay {sample} "+Params["Run"]+" with uproot")
            NuMI_MC_signal = uproot3.open("../NuMI_signal/KDAR_dump/sfnues/pi0/DetVars/"+f"{HNL_mass}_{DetVar}_"+Params["Run"]+".root")[root_dir+"/"+main_tree]
            df_signal = NuMI_MC_signal.pandas.df(['run','sub','evt'], flatten=Params["FLATTEN"])
            file = df_signal
            Functions.make_unique_ev_id(file) #This creates "rse_id" branch
            new_signal = file.copy()
            df_rse_list.append(new_signal)

        common_evs = Functions.make_common_evs_df(df_rse_list)
        common_evs_dict[HNL_mass] = common_evs
else: common_evs = None

Loading only run, subrun, event numbers

150


/home/david/HNL/ee_decays/Utilities/Functions.py:180: FutureWarning: Passing 'suffixes' which cause duplicate columns {'run_x', 'evt_x', 'sub_x'} in the result is deprecated and will raise a MergeError in a future version.
  overlapping_df = functools.reduce(lambda left,right: pd.merge(left, right, on=['rse_id'], how='inner'), df_list)


Length is of common events list is 9159
180
Length is of common events list is 8672
200
Length is of common events list is 0
220
Length is of common events list is 8732
240
Length is of common events list is 9323
245
Length is of common events list is 9010


## Making new variables

In [7]:
def Make_first_new_variables(df): #Only for variables which can be directly made from the ntuple variables with no manipulation/selection
    df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']
    df['trkfit'] = df['shr_tkfit_npointsvalid'] / df['shr_tkfit_npoints']
    #Something like shr_tkfit_distance
    return df.copy

#Works, but takes a while, ignored objects with huge negative numbers for the positions.
def Make_fiducial_vars(df):
    print(len(df))
    n_pfps = df["n_pfps"].groupby(level="entry").apply(max)
    df_placeholder = df.query("trk_sce_start_x_v>-1e4").copy()
    print(len(df_placeholder))
    min_x=df_placeholder[["trk_sce_start_x_v","trk_sce_end_x_v"]].min(axis=1).groupby(level="entry").apply(min)
    max_x=df_placeholder[["trk_sce_start_x_v","trk_sce_end_x_v"]].max(axis=1).groupby(level="entry").apply(max)
    
    min_y=df_placeholder[["trk_sce_start_y_v","trk_sce_end_y_v"]].min(axis=1).groupby(level="entry").apply(min)
    max_y=df_placeholder[["trk_sce_start_y_v","trk_sce_end_y_v"]].max(axis=1).groupby(level="entry").apply(max)
    
    min_z=df_placeholder[["trk_sce_start_z_v","trk_sce_end_z_v"]].min(axis=1).groupby(level="entry").apply(min)
    max_z=df_placeholder[["trk_sce_start_z_v","trk_sce_end_z_v"]].max(axis=1).groupby(level="entry").apply(max)
    
    del df_placeholder
    
    df2 = df.copy()
    
    df2["min_x"]=np.array(np.repeat(min_x, np.array(n_pfps)))
    df2["max_x"]=np.array(np.repeat(max_x, np.array(n_pfps)))
    df2["min_y"]=np.array(np.repeat(min_y, np.array(n_pfps)))
    df2["max_y"]=np.array(np.repeat(max_y, np.array(n_pfps)))
    df2["min_z"]=np.array(np.repeat(min_z, np.array(n_pfps)))
    df2["max_z"]=np.array(np.repeat(max_z, np.array(n_pfps)))
    return df2

# Reading in and pickling files

In [9]:
if ((Params["Load_Signal_DetVars"] == True)or(Params["Load_pi0_signal_DetVars"] == True)) and (Params["Only_keep_common_DetVar_evs"] == True):
    Functions.Load_and_pkl_samples(samples, sample_loc, loc_pkls, common_evs_dict, Params, save_str="_FINAL")
else:
    Functions.Load_and_pkl_samples(samples, sample_loc, loc_pkls, common_evs, Params, save_str="_FINAL") #Remove "_New_gen" str for data
print("Done!")


Done!


## Testing Make_fiducial_vars error

In [ ]:
NuMI_MC_overlay=uproot3.open(f"../NuMI_MC/DetVars/neutrinoselection_filt_"+Params["Run"]+f"_overlay_WireModThetaXZ.root")[Constants.root_dir+"/"+Constants.main_tree]
df_overlay = NuMI_MC_overlay.pandas.df(Params["variables_MC"], flatten=Params["FLATTEN"])


In [ ]:
len(df_overlay)

In [ ]:
loc_pkls = "pkl_files/"+Params["Run"]+"/current_files/"+Params["variables_string"]+"/"
file = pd.read_pickle(loc_pkls+f"DetVars/overlay_run3_my_vars_WireModThetaXZ_flattened.pkl")
red_file = pd.read_pickle(loc_pkls+f"DetVars/overlay_run3_my_vars_WireModThetaXZ_flattened_reduced_evs.pkl")
new_file = pd.read_pickle(loc_pkls+f"DetVars/overlay_run3_my_vars_WireModThetaXZ_flattened_reduced_evs_FINAL.pkl")

print(len(file))
print(len(red_file))
print(len(new_file))

In [ ]:
# df_test = Functions.Make_fiducial_vars(df_overlay)
n_pfps = df_overlay["n_pfps"].groupby(level="entry").apply(max)
df_placeholder = df_overlay.query("trk_sce_start_x_v>-1e4").copy()
n_pfps_2 = df_placeholder["n_pfps"].groupby(level="entry").apply(max)
print("Length df " + str(len(df_overlay)))
print("Length n_pfps " + str(len(n_pfps)))
print("Length n_pfps_2 " + str(len(n_pfps_2)))
print("Length placeholder " + str(len(df_overlay)))

min_x=df_placeholder[["trk_sce_start_x_v","trk_sce_end_x_v"]].min(axis=1).groupby(level="entry").apply(min)
max_x=df_placeholder[["trk_sce_start_x_v","trk_sce_end_x_v"]].max(axis=1).groupby(level="entry").apply(max)
min_y=df_placeholder[["trk_sce_start_y_v","trk_sce_end_y_v"]].min(axis=1).groupby(level="entry").apply(min)
print("Length min_x " + str(len(min_x)))
print("Length max_x " + str(len(max_x)))
print("Length min_y " + str(len(min_y)))

In [ ]:
# 65535
n_pfps.head(20)
pfp_ids = n_pfps.index.values
min_x_ids = min_x.index.values

for val in pfp_ids:
    if val not in min_x_ids:
        print(val)

In [ ]:
n_pfps.index

In [ ]:
n_pfps[65535]

In [ ]:
test = df_overlay.loc[[65535]]
test_placeholder = df_placeholder.loc[[65535]]

In [ ]:
test.keys()

In [ ]:
test['trk_sce_start_x_v']

# Finished code

In [ ]:
New_feature_list = ['shrclusdir2', 'n_tracks', 'trk_energy', 'shr_theta_v', 'contained_sps_ratio', 'trk_chipr_best', 'shr_px_v',
                    'trk_end_x_v', 'n_pfps', 'pfnplanehits_V', 'pfnplanehits_U', 'trk_calo_energy_u_v', 'nu_flashmatch_score', 'trk_score_v',
                    'NeutrinoEnergy2', 'shr_phi_v', 'pfnplanehits_Y', 'shr_pz_v', 'trk_theta_v', 'trk_phi_v', 'trk_energy_hits_tot',
                    'trk_dir_z_v', 'SliceCaloEnergy2'] 

Preselection_vars = ["nslice","flash_time","nu_flashmatch_score","NeutrinoEnergy2","contained_fraction","trk_score","trk_score_v","n_pfps"]

non_duplicate_vars = []

for var in Preselection_vars:
    if var not in New_feature_list:
        non_duplicate_vars.append(var)

print(non_duplicate_vars)

In [ ]:
root_dir = 'nuselection'
main_tree = 'NeutrinoSelectionFilter'
loc_pkls = f"pkl_files/{Run}/current_files/{variables_string}/"

for sample in samples: #Looping over all samples, should make a function for this.
    if sample in Constants.Detector_variations: #Checks if it is an overlay Detector Variation sample
        print(f"Loading overlay {sample} {Run} with uproot")
        NuMI_MC_overlay = uproot3.open(f'../NuMI_MC/DetVars/neutrinoselection_filt_{Run}_overlay_{sample}.root')[root_dir+'/'+main_tree]
        df_overlay = NuMI_MC_overlay.pandas.df(variables_MC, flatten=FLATTEN)
        file = df_overlay
        Functions.Edit_Weight_Tune(file)
        Functions.MC_weight_branch(file)
        Functions.make_unique_ev_id(file) #This creates "rse_id" branch
        if Only_keep_common_DetVar_evs == True:
            filtered = file.loc[(file['rse_id'].isin(common_evs['rse_id']))]
            new_overlay = filtered.copy()
            del(file)
            del(filtered)
        else:
            new_overlay = file.copy()
            del(file)
        print(f"Pickling {Run} overlay {sample} file")
        new_overlay.to_pickle(loc_pkls+f"DetVars/overlay_{Run}_{variables_string}_{sample}_{Flat_state}_{Reduced_state}.pkl")
        del(new_overlay)
    else: #Standard sample types
        print(f"Loading {sample} {Run} file with uproot")
        if Constants.sample_type[sample] == "MC_signal":
            for HNL_mass in Constants.HNL_mass_samples:
                uproot_file = uproot3.open(sample_loc[sample]+f'{HNL_mass}_Umu4_majorana_numi_{current}.root')[root_dir+'/'+main_tree]
                df_signal = uproot_file.pandas.df(variables, flatten=FLATTEN)
                file = df_signal
                new_signal = file.copy()
                del(file)
                print(f"Pickling {Run} {HNL_mass}MeV file")
                new_signal.to_pickle(loc_pkls+f"signal_{HNL_mass}MeV_{Run}_{variables_string}_"+Flat_state+".pkl")
                del(new_signal)
        else:
            uproot_file = uproot3.open(sample_loc[sample])[root_dir+'/'+main_tree]
            if Constants.sample_type[sample] == "MC":
                df = uproot_file.pandas.df(variables_MC, flatten=FLATTEN)
                file = df
                Functions.Edit_Weight_Tune(file)
                Functions.MC_weight_branch(file)
            else:
                df = uproot_file.pandas.df(variables, flatten=FLATTEN)
                file = df
            new_file = file.copy()
            del(file)
            print(f"Pickling {Run} {sample} file")
            new_file.to_pickle(loc_pkls+f"{sample}_{Run}_{variables_string}_"+Flat_state+".pkl")
            del(new_file)

print("\n"+"Finished")

In [ ]:
#Testing loading reweighting branches
sys_variables = Variables.sys_vars + Variables.event_vars + Variables.weight_related

NuMI_MC_overlay = uproot3.open(loc_overlay_run1)[root_dir+'/'+main_tree]
df_overlay = NuMI_MC_overlay.pandas.df(sys_variables, flatten=False)


In [ ]:
overlay_from_pkl = pd.read_pickle(loc_pkls+"overlay_run1_my_vars_flattened.pkl")

Preselection_efficiency_signal = [1.0]

Preselection_with_list(overlay_from_pkl, Constants.Preselection_dict, Preselection_efficiency_signal)

In [ ]:
print(len(overlay_from_pkl))

# Plotting Variables before preselection

In [ ]:
My_vars = Variables.First_pass_vars
My_vars_MC = Variables.First_pass_vars_MC

overlay_from_pkl = pd.read_pickle(loc_pkls+"overlay_run1_my_vars1_flattened.pkl")
dirt_from_pkl = pd.read_pickle(loc_pkls+"dirt_run1_my_vars1_flattened.pkl")
EXT_from_pkl = pd.read_pickle(loc_pkls+"EXT_run1_my_vars1_flattened.pkl")
signal_from_pkl = pd.read_pickle(loc_pkls+f"signal_100MeV_run1_my_vars1_flattened.pkl")

In [ ]:
samples = {"overlay":overlay_from_pkl,
           "dirtoverlay":dirt_from_pkl,
           "beamoff":EXT_from_pkl,
           "signal":signal_from_pkl}
sample_norms = {"overlay":SF_overlay_run1,
           "dirtoverlay":SF_dirt_run1,
           "beamoff":SF_EXT_run1,
           "signal":SF_signal_run1}

PT.Plot_preselection_variable("n_pfps", samples, sample_norms, xlabel=[],xlims=[1,8],bins=7,figsize=[8,8],HNLplotscale=10000000,logy="False")

## Load in all branches for small number of events

In [ ]:
NuMI_MC_dirt_run1 = uproot3.open(loc_dirt_run1)[root_dir+'/'+main_tree]
NuMI_EXT_run1 = uproot3.open(loc_EXT_run1)[root_dir+'/'+main_tree]
#NuMI_beamgood_run1 = uproot3.open(loc_beamgood_run1)[root_dir+'/'+main_tree]
Signal_run1 = uproot3.open(loc_signal_run1)[root_dir+'/'+main_tree]

print("----MC OVERLAY BACKGROUND----")
print("Number of branches is " + str(len(NuMI_MC_overlay_run1.keys()))) 
print("Number of events is " + str(NuMI_MC_overlay_run1.numentries))
print("----MC DIRT BACKGROUND----")
print("Number of branches is " + str(len(NuMI_MC_dirt_run1.keys()))) 
print("Number of events is " + str(NuMI_MC_dirt_run1.numentries))
print("----EXT BACKGROUND----")
print("Number of branches is " + str(len(NuMI_EXT_run1.keys()))) 
print("Number of events is " + str(NuMI_EXT_run1.numentries))
# print("----DATA----")
# print("Number of branches is " + str(len(NuMI_beamgood_run1.keys()))) 
# print("Number of events is " + str(NuMI_beamgood_run1.numentries))
print("----SIGNAL----")
print("Number of branches is " + str(len(Signal_run1.keys()))) 
print("Number of events is " + str(Signal_run1.numentries))

NumEvents = 10
FLATTEN = False

df_overlay_run1 = NuMI_MC_overlay_run1.pandas.df("*", entrystop=NumEvents, flatten=FLATTEN)
df_dirt_run1 = NuMI_MC_dirt_run1.pandas.df("*", entrystop=NumEvents, flatten=FLATTEN)
df_EXT_run1 = NuMI_EXT_run1.pandas.df("*", entrystop=NumEvents, flatten=FLATTEN)
df_signal_run1 = Signal_run1.pandas.df("*", entrystop=NumEvents, flatten=FLATTEN)

overlay_vars = df_overlay_run1.keys()
dirt_vars = df_dirt_run1.keys()
EXT_vars = df_EXT_run1.keys()
signal_vars = df_signal_run1.keys()

In [ ]:
# for i in overlay_vars:
#     print(i)
    
overlay_no_truth_vars = []
    
for j in overlay_vars:
    if 'true' in j:
        continue
    if 'truth' in j:
        continue
    if 'mc_' in j:
        continue
    overlay_no_truth_vars.append(j)
    #print(j)

print(len(overlay_no_truth_vars))

# Testing other ways of saving .pkls

In [ ]:
def make_common_evs_list(df_list): #Takes a list of dataframes
    common_events = df_list[0]['rse_id'].to_list()
    for df in df_list:
        temp_rse_list = df['rse_id'].to_list()
        for entry in common_events: #Looping over all events in the dataframe
            if entry not in temp_rse_list:
                common_events.remove(entry)
        print("Done one") 
    print("Number of common events is " + str(len(common_events)))
    return common_events

def only_keep_evs_in_list(df, ev_list):
    print("Length of dataframe is " + str(len(df)))
    if pd.Series(['rse_id']).isin(df.columns).all():
        for entry in df.index: #Looping over all events in the dataframe
            if df['rse_id'][entry] not in ev_list:
                df.drop(entry)
        print("Length after dropping events is " + str(len(df)))
    else:
        print("Dataframe needs \"rse_id\" column.")
        return 0

In [ ]:
# run1_samples = [overlay_from_pkl, dirt_from_pkl, EXT_from_pkl, signal_from_pkl]
# run1_sample_names = ["df_overlay_run1", "df_dirt_run1", "df_EXT_run1", "df_signal_run1"] #Better way to do this, dictionary?

# loc_pkls = "pkl_files/"
# i = 0

# for sample in run1_samples:
#     sample.to_pickle(loc_pkls+run1_sample_names[i]+"_full.pkl")
#     i+=1

In [ ]:
# flattened_overlay_from_pkl = pd.read_pickle(loc_pkls+"overlay_run1_Aditya_vars_flattened.pkl")

# NumEvs_overlay_run1 = 914728 #Should have this saved somewhere, or perhaps accessed direcly from uproot? 

# print(flattened_overlay_from_pkl['nslice'])
# print(flattened_overlay_from_pkl['nslice'][0])

# number_evs_removed = 0
# number_evs_kept = 0
# for entry in range(NumEvs_overlay_run1):
#     #if entry in flattened_overlay_from_pkl['nslice']:
#     if entry in flattened_overlay_from_pkl.index:
#         number_evs_kept += 1
#     else:
#         number_evs_removed += 1

# print("Number of events kept is " + str(number_evs_kept))
# print("Number of events removed is " + str(number_evs_removed))



In [ ]:
#Fraction with nslice of 1 is 0.41223401929316694
fraction_reco = number_evs_kept/(number_evs_kept+number_evs_removed)
print("Fraction with nslice of 1 is " + str(fraction_reco))